<img style="float: left;" src="http://3.bp.blogspot.com/-cRW46iETTJw/VPIc1vbEWWI/AAAAAAAAAUU/3Hdq70uDl6U/s1600/il%2Bmare%2Bdi%2Bghiaccio%2B(1).jpg" width="700px"/>

***The Sea of Ice - Caspar David Friedrich - 1824***

## ***Titanic***

- EDA
- Classification Pipeline
- RandomGridCV
- ANN

In [1]:
import warnings  
warnings.filterwarnings('ignore')
import os
import numpy as np
import pandas as pd
import math 
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import export_graphviz 
from sklearn.metrics import roc_curve, auc 
from sklearn.metrics import classification_report 
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [1]:
dataset = pd.read_csv('/kaggle/input/titanic/train.csv')

In [1]:
dataset.describe()

In [1]:
dataset.drop(['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True) 


In [1]:
dataset.isnull().sum()

In [1]:
dataset['Age'].fillna(dataset['Age'].median(), inplace = True)

dataset["Embarked"] = dataset["Embarked"].fillna("S")

dataset.isnull().sum()

***Survived***

In [1]:
sns.countplot(x='Survived', hue='Sex', data=dataset)

***SibSp***

In [1]:
sns.countplot(x='SibSp', hue='Survived', data=dataset)

***Parch***

In [1]:
sns.countplot('Parch' , hue = 'Survived' , data = dataset)

***Embarked***

In [1]:
sns.countplot(x='Embarked', hue='Survived', data=dataset)

***Pclass***

In [1]:
sns.countplot(x='Pclass', hue='Survived', data=dataset)

In [1]:
dataset = pd.get_dummies(dataset)

In [1]:
y = dataset['Survived'].values 

X = dataset.drop(['Survived'],axis=1)                 


In [1]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)


from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [1]:

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB



classifiers =  [
       ['Logistic Regression Classifier :', LogisticRegression()] ,
       ['Decision Tree Classifier :', DecisionTreeClassifier()] ,
       ['Random Forest Classifier :', RandomForestClassifier()] ,
       ['Gradient Boosting Classifier :', GradientBoostingClassifier()] ,
       ['AdaBoost Classifier :', AdaBoostClassifier()] ,
       ['XGB Classifier :', XGBClassifier()] ,
       ['Extra Tree Classifier :', ExtraTreesClassifier()] ,
       ['K-Neighbors Classifier :', KNeighborsClassifier()] ,  
       ['Naive Bayes :' , GaussianNB()] ,
       ]


confusion_tab = []

for name,model in classifiers:    

    model = model
    
    model.fit(X_train,y_train)
    
    y_pred = model.predict(X_test)
        

    print('-----------------------------------')
    print(name)
    print('Accuracy Score ', accuracy_score( y_test, y_pred))
    print("f1_score: ",f1_score( y_test, y_pred))
    print("precision_score: ", precision_score( y_test, y_pred))
    print("recall_score: ", recall_score( y_test, y_pred))
    print("ROC AUC: ", roc_auc_score( y_test, y_pred))
    print('---------------------------------')
    confusion = confusion_matrix( y_test, y_pred)
    confusion_tab.append(confusion)    
    plt.subplot(2,2,1)
    plt.title(name)
    sns.heatmap(confusion, annot=True , cmap="Blues" , fmt="d" , cbar=False , annot_kws = {"size": 24})
    plt.show()



Best Model  --> ExtraTreeClassifier

In [1]:
n_estimators = [int(x) for x in np.linspace(start = 1, stop = 100, num = 10 )]  
max_depth = [int(x) for x in np.linspace(1, 100, num = 10)] 
max_depth.append(None)

min_samples_split = [2, 5, 10]     
min_samples_leaf = [1, 2, 4]       
max_features = ['auto', 'sqrt' , 'log2']    
bootstrap = [True, False]         
random_state = [42]                





param_distributions = dict(n_estimators = n_estimators,
                   max_depth = max_depth, 
                   min_samples_split = min_samples_split,
                   min_samples_leaf = min_samples_leaf,
                   max_features = max_features, 
                   bootstrap = bootstrap,
                   random_state = random_state ,
                   )




estimator = ExtraTreesClassifier()     

                         
from sklearn.model_selection import RandomizedSearchCV




RandomCV = RandomizedSearchCV(estimator=estimator,          
                            param_distributions=param_distributions,
                            n_iter = 50,
                            cv = 10,
                            scoring = 'precision',
                            verbose=1, 
                            n_jobs=-1,
                            )



hyper_model = RandomCV.fit(X_train, y_train)

print('Best Score: ', hyper_model.best_score_)  
print('Best Params: ', hyper_model.best_params_)

In [1]:
hyper_model.best_estimator_.fit(X_train,y_train) 

y_pred_hyper = hyper_model.best_estimator_.predict(X_test) 


print('Accuracy Score ', accuracy_score( y_test, y_pred_hyper))
print("f1_score: ",f1_score( y_test, y_pred_hyper))
print("precision_score: ", precision_score( y_test, y_pred_hyper))
print("recall_score: ", recall_score( y_test, y_pred_hyper))
print("ROC AUC: ", roc_auc_score( y_test, y_pred_hyper))

confusion_hyper = confusion_matrix(y_test ,y_pred_hyper )

plt.subplot(2,3,1)
plt.title("Hyper Model Confusion Matrix")
sns.heatmap(confusion_hyper,annot=True,fmt="d",cbar=False, annot_kws={"size": 24})

# ***ANN***

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense

classifier = Sequential()

classifier.add(Dense(units = 20, kernel_initializer = 'uniform', activation = 'relu', input_dim = 10))

classifier.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'relu'))

classifier.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'relu'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))


classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


classifier.fit(X_train, y_train, batch_size = 1, epochs = 200)  

In [1]:
y_pred_ANN = classifier.predict(X_test)

y_pred_ANN = [1 if each > 0.5 else 0 for each in y_pred_ANN]
y_pred_ANN = np.array(y_pred_ANN)

  
print('Accuracy Score ', accuracy_score( y_test, y_pred_ANN))
print("f1_score: ",f1_score( y_test, y_pred_ANN))
print("precision_score: ", precision_score( y_test, y_pred_ANN))
print("recall_score: ", recall_score( y_test, y_pred_ANN))
print("ROC AUC: ", roc_auc_score( y_test, y_pred_ANN))

confusion_ANN = confusion_matrix(y_test, y_pred_ANN)   

plt.subplot(2,3,1)
plt.title("ANN Confusion Matrix")
sns.heatmap(confusion_ANN,annot=True,fmt="d",cbar=False, annot_kws={"size": 24})